In [24]:
# librairies loading

import kagglehub
import os

import re
import nltk
import numpy as np
import pandas as pd



# 1-) Preprocessing

In [3]:
# load dataset

path = kagglehub.dataset_download("tobiasbueck/multilingual-customer-support-tickets")
print(path)

C:\Users\mayen\.cache\kagglehub\datasets\tobiasbueck\multilingual-customer-support-tickets\versions\12


In [4]:
for root, dirs, files in os.walk(path):
    for file in files:
        print(f"Found file: {file}")

Found file: aa_dataset-tickets-multi-lang-5-2-50-version.csv
Found file: dataset-tickets-german_normalized.csv
Found file: dataset-tickets-german_normalized_50_5_2.csv
Found file: dataset-tickets-multi-lang-4-20k.csv
Found file: dataset-tickets-multi-lang3-4k.csv


In [4]:
data_path = os.path.join(path , "aa_dataset-tickets-multi-lang-5-2-50-version.csv")
data = pd.read_csv(data_path)
data.head()

# save data in another folder
# os.getcwd()
# data.to_csv(os.path.join(os.getcwd(), "../data/raw_data/raw_data.csv"))

,subject,body,answer,type,queue,priority,language,version,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8
0,Wesentlicher Sicherheitsvorfall,"Sehr geehrtes Support-Team,\n\nich möchte eine...",Vielen Dank für die Meldung des kritischen Sic...,Incident,Technical Support,high,de,51,Security,Outage,Disruption,Data Breach,NaN,NaN,NaN,NaN
1,Account Disruption,"Dear Customer Support Team,\n\nI am writing to...","Thank you for reaching out, <name>. We are awa...",Incident,Technical Support,high,en,51,Account,Disruption,Outage,IT,Tech Support,NaN,NaN,NaN
2,Query About Smart Home System Integration Feat...,"Dear Customer Support Team,\n\nI hope this mes...",Thank you for your inquiry. Our products suppo...,Request,Returns and Exchanges,medium,en,51,Product,Feature,Tech Support,NaN,NaN,NaN,NaN,NaN
3,Inquiry Regarding Invoice Details,"Dear Customer Support Team,\n\nI hope this mes...",We appreciate you reaching out with your billi...,Request,Billing and Payments,low,en,51,Billing,Payment,Account,Documentation,Feedback,NaN,NaN,NaN
4,Question About Marketing Agency Software Compa...,"Dear Support Team,\n\nI hope this message reac...",Thank you for your inquiry. Our product suppor...,Problem,Sales and Pre-Sales,medium,en,51,Product,Feature,Feedback,Tech Support,NaN,NaN,NaN,NaN


In [14]:
chainea = "maman"
chaineb = "papa"
chainea+'_'+chaineb

'maman_papa'

In [17]:
selected_columns = ['subject','body', 'answer','type','queue','priority']
data_clean1 = data[selected_columns]

data_clean1["topic"] = data_clean1["subject"] + ' ' + data_clean1["body"] 
data_clean1["type_and_priority"] = data_clean1["type"] + '_' + data_clean1["priority"] 

data_clean1 = data_clean1.drop(columns = ["subject", "body", "type", "priority"], axis = 1)
data_clean1.head()

C:\Users\mayen\AppData\Local\Temp\ipykernel_88912\1014220828.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean1["topic"] = data_clean1["subject"] + ' ' + data_clean1["body"]
C:\Users\mayen\AppData\Local\Temp\ipykernel_88912\1014220828.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean1["type_and_priority"] = data_clean1["type"] + '_' + data_clean1["priority"]


,answer,queue,topic,type_and_priority
0,Vielen Dank für die Meldung des kritischen Sic...,Technical Support,Wesentlicher Sicherheitsvorfall Sehr geehrtes ...,Incident_high
1,"Thank you for reaching out, <name>. We are awa...",Technical Support,"Account Disruption Dear Customer Support Team,...",Incident_high
2,Thank you for your inquiry. Our products suppo...,Returns and Exchanges,Query About Smart Home System Integration Feat...,Request_medium
3,We appreciate you reaching out with your billi...,Billing and Payments,Inquiry Regarding Invoice Details Dear Custome...,Request_low
4,Thank you for your inquiry. Our product suppor...,Sales and Pre-Sales,Question About Marketing Agency Software Compa...,Problem_medium


In [18]:
data_clean1["type_and_priority"].value_counts()

type_and_priority
Incident_high      4931
Incident_medium    4373
Request_medium     3539
Request_high       2727
Problem_medium     2422
Problem_high       2351
Incident_low       2162
Request_low        1921
Problem_low        1239
Change_medium      1181
Change_high        1169
Change_low          572
Name: count, dtype: int64

## 1-b ) cleaning and translation

In [29]:
def basic_cleaning(text : str) -> str :
    text = text.strip()
    text = re.sub(r"\n+", " ",  text)
    text = re.sub(r"http\S+|www.\S+", "",text)
    text = re.sub(r"[^A-Za-zÀ-ÿ0-9\s.,!?%$€@\-_/]", "",text)
    text = re.sub(r"\s+", " ", text)

    return text.strip()

data_clean1["topic"] = data_clean1["topic"].apply(lambda x : basic_cleaning(str(x)))
data_clean1["answer"] = data_clean1["answer"].apply(lambda x : basic_cleaning(str(x)))

In [30]:
# changed observations
print(data_clean1["topic"][0])
print(data_clean1["topic"][0])


,answer,queue,topic,type_and_priority
0,Vielen Dank für die Meldung des kritischen Sic...,Technical Support,Wesentlicher Sicherheitsvorfall Sehr geehrtes ...,Incident_high
1,"Thank you for reaching out, name. We are aware...",Technical Support,"Account Disruption Dear Customer Support Team,...",Incident_high
2,Thank you for your inquiry. Our products suppo...,Returns and Exchanges,Query About Smart Home System Integration Feat...,Request_medium
3,We appreciate you reaching out with your billi...,Billing and Payments,Inquiry Regarding Invoice Details Dear Custome...,Request_low
4,Thank you for your inquiry. Our product suppor...,Sales and Pre-Sales,Question About Marketing Agency Software Compa...,Problem_medium
